## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

d:\Python3.10\lib\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        super(myModel, self).__init__()
        self.W1 = tf.Variable(tf.random.normal([28*28, 14*14]))
        self.b1 = tf.Variable(tf.zeros([14*14]))
        self.W2 = tf.Variable(tf.random.normal([14*14, 49]))
        self.b2 = tf.Variable(tf.zeros([49]))
        ####################

    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        flat_x = tf.reshape(x, shape=[-1, 28*28])
        tmp_x = tf.matmul(flat_x, self.W1) + self.b1
        logits = tf.matmul(tmp_x, self.W2) + self.b2
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [5]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 318.69473 ; accuracy 0.0015333333
epoch 1 : loss 294.7839 ; accuracy 0.00245
epoch 2 : loss 273.765 ; accuracy 0.0043166666
epoch 3 : loss 254.86037 ; accuracy 0.0072333333
epoch 4 : loss 237.80647 ; accuracy 0.010983333
epoch 5 : loss 222.3723 ; accuracy 0.0157
epoch 6 : loss 208.39343 ; accuracy 0.02135
epoch 7 : loss 195.72684 ; accuracy 0.027966667
epoch 8 : loss 184.24907 ; accuracy 0.035566665
epoch 9 : loss 173.81866 ; accuracy 0.043283332
epoch 10 : loss 164.29564 ; accuracy 0.051283333
epoch 11 : loss 155.6019 ; accuracy 0.0603
epoch 12 : loss 147.65257 ; accuracy 0.07038333
epoch 13 : loss 140.35762 ; accuracy 0.079833336
epoch 14 : loss 133.6527 ; accuracy 0.09025
epoch 15 : loss 127.4668 ; accuracy 0.1004
epoch 16 : loss 121.72938 ; accuracy 0.111216664
epoch 17 : loss 116.39854 ; accuracy 0.1226
epoch 18 : loss 111.43238 ; accuracy 0.13408333
epoch 19 : loss 106.80765 ; accuracy 0.14626667
epoch 20 : loss 102.491035 ; accuracy 0.15928334
epoch 21 : loss 98.4